In [1]:
# Importings
import pyomo.environ as pyo
import os
import numpy as np
import pandas as pd
import math

solverpath_exe = 'C:\\Users\\Yuri Muniz\\OneDrive\\Documentos\\solvers\\ipopt-win64\\ipopt'

### Decision variables:

$$-10 \le p_i \le 20,$$
where $p_i$ is the percentage change in the price of product $i$.

### Objective: 

$$\max \mbox{profit}(R,\{S_i\},\{P_i\}) = R\sum_i S_i(P_i - C_i),$$

where

$$S_i = S_{0i}\times (1 - I_{vi}p_i),$$

$$P_i = P_{0i}\times(1 + p_i/100),$$

$$R = R_0\times \prod_i\left(1 - I_{ai}p_i\right),$$

and all else are input parameters given by

- $S_{0i}$: average units sold of product $i$ in the initial configuration of prices;
- $P_{0i}$: average price per unit sold of product $i$ in the initial configuration of prices;
- $C_i$: cost of product $i$ (assumed to be constant);
- $R_0 = 0.05$: initial customer response rate (5%);
- $I_{vi}$: increase sale volume per product. How much more sales we get of a product if we decrease its price by 1%;
- $I_{ai}$: Incremental acquisition of product $i$. How much more customers respond to our whole catalog of products; if we decrease the price of product $i$ by 1%.

### Ref.: 
https://www.analyticsvidhya.com/blog/2016/07/solving-case-study-optimize-products-price-online-vendor-level-hard/

### Useful links to understand the variabes:
https://www.repsly.com/blog/consumer-goods/how-to-calculate-sales-volume-use-it-to-win-in-field

In [2]:
# Dataset from ref.
prod_df = pd.read_csv('Vendor_data.csv')
prod_df.head()

,Products,Avg_Price_per_unit,Cost_per_unit,Average_Profit_per_unit,Average_units_sold,Incremental_acquisition,Increase_sale_volume
0,Product 1,95,92,2.8,0.6,0.00323,0.08906
1,Product 2,37,27,10.0,0.8,0.00106,0.01991
2,Product 3,34,26,8.5,0.6,0.00328,0.03778
3,Product 4,32,30,2.8,0.7,0.00000,0.08562
4,Product 5,62,45,16.0,1.0,0.00079,0.02637


In [3]:
# Renaming columns according to the problem description
cols_dict = {'Avg_Price_per_unit': 'P0', 
'Cost_per_unit': 'C',
'Average_units_sold':'S0', 
'Incremental_acquisition': 'Ia',
'Increase_sale_volume':'Iv'}
prod_df = prod_df.rename(columns=cols_dict)
prod_df = prod_df[list(cols_dict.values())]

# Changing the scale of Ia and Iv to be associate to a decrease of 1% instead of 10%
prod_df['Ia'] = prod_df['Ia']/10
prod_df['Iv'] = prod_df['Iv']/10

prod_df.head()

,P0,C,S0,Ia,Iv
0,95,92,0.6,0.000323,0.008906
1,37,27,0.8,0.000106,0.001991
2,34,26,0.6,0.000328,0.003778
3,32,30,0.7,0.000000,0.008562
4,62,45,1.0,0.000079,0.002637


In [4]:
R0 = 0.05
profit0 = (prod_df['S0']*(prod_df['P0'] - prod_df['C'])).sum()*R0
print('profit prior optimization: '+ str(round(profit0,1)))

profit prior optimization: 164.3


In [5]:
model = pyo.AbstractModel()

# Params
model.N = pyo.Param()
model.i = pyo.RangeSet(1,model.N)
model.P0 = pyo.Param(model.i)
model.C = pyo.Param(model.i)
model.S0 = pyo.Param(model.i)
model.R0 = pyo.Param()
model.Ia = pyo.Param(model.i)
model.Iv = pyo.Param(model.i)

# variable (price percentage change)
model.p = pyo.Var(model.i, bounds = (-10,20), within = pyo.Reals, initialize = 0)

# auxiliar variables
model.R = pyo.Var()
def R_constraint(model):
    return model.R == model.R0*math.prod([1 - model.p[i]*model.Ia[i] for i in model.i])
model.RC = pyo.Constraint(rule = R_constraint)

model.S = pyo.Var(model.i, within = pyo.NonNegativeReals)
def S_constraint(model,i):
    return model.S[i] == model.S0[i]*(1 - model.Iv[i]*model.p[i])
model.SC = pyo.Constraint(model.i, rule = S_constraint)

model.P = pyo.Var(model.i, within = pyo.NonNegativeReals)
def P_constraint(model,i):
    return model.P[i] == model.P0[i]*(1 + model.p[i]/100)
model.PC = pyo.Constraint(model.i, rule = P_constraint)

# Objective
model.OF = pyo.Var()
def OF_constraint(model):
    return model.OF == model.R*sum(model.S[s]*(model.P[s] - model.C[s]) for s in model.i)
model.OFC = pyo.Constraint(rule = OF_constraint)

model.obj = pyo.Objective(expr = model.OF, sense = pyo.maximize)

# ipopt solver
opt = pyo.SolverFactory('ipopt', executable=solverpath_exe)

In [6]:
N = 250

input_dict =  {None:
    {'N':{None:N},
     'P0':{i+1:v for i,v in enumerate(prod_df['P0'][:N].tolist())},
     'C':{i+1:v for i,v in enumerate(prod_df['C'][:N].tolist())},
     'S0':{i+1:v for i,v in enumerate(prod_df['S0'][:N].tolist())},
     'R0':{None: 0.05},
     'Ia':{i+1:v for i,v in enumerate(prod_df['Ia'][:N].tolist())},
     'Iv':{i+1:v for i,v in enumerate(prod_df['Iv'][:N].tolist())}
    }
                }

instance = model.create_instance(input_dict)
results = opt.solve(instance)
print(f'optimized profit: {round(pyo.value(instance.OF),0)}')

optimized profit: 308.0


In [7]:
print('20th first product price percentage changes:\n')
for i in range(1,21):
    print(round(pyo.value(instance.p[i]),1))

20th first product price percentage changes:

-10.0
-10.0
-10.0
20.0
20.0
20.0
20.0
-10.0
-10.0
20.0
-10.0
20.0
-10.0
-10.0
20.0
-10.0
20.0
-4.1
-10.0
-10.0
